In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
#Create New Target (Good in 60-15 but bad in 90-24)
model_data['G15_B24'] = model_data.apply(
   lambda row: 0 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 0)
    else ( 1 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 1)
    else 2),axis=1)

model_data= model_data[model_data['G15_B24']!=2]
print('********* New Target Prepared *********')

********* New Target Prepared *********


In [4]:
model_data['G15_B24'].value_counts(dropna=False)

G15_B24
0    44460
1     3172
Name: count, dtype: int64

In [5]:
# take one copy for model development data
data = model_data.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [6]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(34344, 2832) (10802, 2832)


In [7]:
features_list=[
    'bounced_transactions_count',
'Min_Balance_Min_M1_M2_M3',
'utilities_spending_pct',
'cash_withdrawal_sum',
'upi_trans_count',
'num_credits_last_1_months',
'ratio_balance_after_2_days_max_credit_M3',
'ratio_6th_day_balance_to_max_balance_M3',
'range_max_credit_last_2_months',
'num_upi_trans_last_3_months',
'ratio_digital_vs_withdrawal',
'balance_5th_day_M1',
'time_max_to_70pct_balance_M2',
'min_max_credit_last_3_months',
'ratio_total_debit_to_total_credit_M2',
'time_between_max_min_balance_M2',
'month_year',
 'G15_B24']

In [8]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', 'G15_B24']) # Adjust 'target' to your actual target column name
y_model = model_df['G15_B24']

X_oot = oot_df.drop(columns=['month_year', 'G15_B24'])
y_oot = oot_df['G15_B24']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_3041569/2552150070.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3041569/2552150070.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [10]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20606, 16), (13738, 16), (20606,), (13738,))

In [11]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(16484, 16) (4122, 16) (13738, 16)


In [12]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [13]:
params = {
    
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.005,
    'max_depth': 2,
    'subsample': 0.3,
    'colsample_bytree': 0.3,
    'lambda': 15,
    'alpha': 5,
    'random_state': 42,
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 100,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.50984	eval-auc:0.51009
[10]	train-auc:0.53070	eval-auc:0.53072
[20]	train-auc:0.53219	eval-auc:0.53571
[30]	train-auc:0.53256	eval-auc:0.53586
[40]	train-auc:0.53227	eval-auc:0.53392
[50]	train-auc:0.53146	eval-auc:0.53273
[60]	train-auc:0.53757	eval-auc:0.53507
[62]	train-auc:0.53768	eval-auc:0.53503


In [14]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.54, Gini: 0.08, KS: 0.07
Eval AUC: 0.54, Gini: 0.07, KS: 0.08
Test AUC: 0.52, Gini: 0.04, KS: 0.04
OOT AUC: 0.55, Gini: 0.10, KS: 0.08


In [15]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
4          0.106383         0.132904        0.147166        0.128832   1645   
3          0.060323         0.127947        0.128825        0.127342   1608   
2          0.054217         0.127206        0.127312        0.127106    664   
1          0.061351         0.126914        0.127052        0.126892  12567   

        label_good  label_bad  
Decile                         
4             1470        175  
3             1511         97  
2              628         36  
1            11796        771

In [16]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
4          0.079470         0.132895        0.147754        0.128825   1359   
3          0.063319         0.127950        0.128825        0.127342   1374   
2          0.070652         0.127209        0.127312        0.127106    552   
1          0.058165         0.126914        0.127052        0.126892  10453   

        label_good  label_bad  
Decile                         
4             1251        108  
3             1287         87  
2              513         39  
1             9845        608

In [17]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
4          0.117484         0.134773        0.147584        0.130174   1081   
3          0.081481         0.128662        0.130168        0.127770   1080   
2          0.090395         0.127362        0.127766        0.127106    885   
1          0.067947         0.126913        0.127052        0.126892   7756   

        label_good  label_bad  
Decile                         
4              954        127  
3              992         88  
2              805         80  
1             7229        527

In [18]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                    Feature  Importance
0                bounced_transactions_count        19.0
4                           upi_trans_count        18.0
14          time_between_max_min_balance_M2        16.0
5                 num_credits_last_1_months        15.0
3                       cash_withdrawal_sum        15.0
8               num_upi_trans_last_3_months        13.0
13     ratio_total_debit_to_total_credit_M2         9.0
7            range_max_credit_last_2_months         8.0
9               ratio_digital_vs_withdrawal         6.0
1                  Min_Balance_Min_M1_M2_M3         4.0
2                    utilities_spending_pct         4.0
6   ratio_6th_day_balance_to_max_balance_M3         3.0
12             min_max_credit_last_3_months         3.0
10                       balance_5th_day_M1         2.0
11             time_max_to_70pct_balance_M2         1.0


In [19]:
feature_importance_df

Feature  Importance
0                bounced_transactions_count        19.0
4                           upi_trans_count        18.0
14          time_between_max_min_balance_M2        16.0
5                 num_credits_last_1_months        15.0
3                       cash_withdrawal_sum        15.0
8               num_upi_trans_last_3_months        13.0
13     ratio_total_debit_to_total_credit_M2         9.0
7            range_max_credit_last_2_months         8.0
9               ratio_digital_vs_withdrawal         6.0
1                  Min_Balance_Min_M1_M2_M3         4.0
2                    utilities_spending_pct         4.0
6   ratio_6th_day_balance_to_max_balance_M3         3.0
12             min_max_credit_last_3_months         3.0
10                       balance_5th_day_M1         2.0
11             time_max_to_70pct_balance_M2         1.0

In [20]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_BA_Train_Pred_1524.csv',index=False)
Eval.to_csv('LeoPayu_BA_Eval_Pred_1524.csv',index=False)
Test.to_csv('LeoPayu_BA_Test_Pred_1524.csv',index=False)
Oot.to_csv('LeoPayu_BA_Oot_Pred_1524.csv',index=False)


In [21]:
#Save the model
xgb_model.save_model('LeoPayu_BA_15_in_24mob.json')
print('Model Saved')

Model Saved
